# Take 2

In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
import plotly.express as px
import statsmodels.api as sm
import pmdarima as pm
import warnings
from sklearn.metrics import mean_squared_error
import math
from statsmodels.tools.sm_exceptions import ConvergenceWarning
warnings.simplefilter('ignore', ConvergenceWarning)

In [73]:
def make_stock_data(stocks):
    info_list=[]
    # Close
    for stock in stocks:
        info = yf.Ticker(stock).history(period='max')[['Close']]
        info_list.append(info)
    combo = pd.concat(info_list,axis=1)
    combo.columns = stocks

    combo = combo.dropna()
    combo.index = pd.to_datetime(combo.index).to_period('D')
    return combo

In [103]:
stock_name = ['pcar','amzn','tsla','oil','aapl']

combo = make_stock_data(stock_name)
combo.head(10)


,pcar,amzn,tsla,oil,aapl
Date,,,,,
2011-04-21,35.873425,9.2945,1.782667,50.349998,10.694243
2011-04-25,35.388565,9.2710,1.759333,50.049999,10.764685
2011-04-26,36.338074,9.1150,1.795333,50.299999,10.685705
2011-04-27,36.573788,9.8315,1.805333,50.810001,10.677468
2011-04-28,36.196671,9.7535,1.844000,50.700001,10.573793
2011-04-29,35.765671,9.7905,1.840000,51.040001,10.676866
2011-05-02,35.745476,10.0595,1.830000,50.950001,10.559460
2011-05-03,35.698326,9.9225,1.791333,49.820000,10.618008
2011-05-04,35.880157,9.9985,1.779333,48.889999,10.659786


In [75]:
def do_model(combo,stock_name):
    outs={}
    for stock in stock_name:
        combo['predict'] = combo[stock].shift(-1)

        model = sm.tsa.statespace.SARIMAX(endog = combo.predict,exog = combo.drop('predict',axis=1),order=[5,0,0],seasonal_order=[0,0,0,5],trend='ct',)
        model_fit = model.fit(disp=False)
        output = model_fit.predict(exog = combo.drop('predict',axis=1))
        combo['predicted'] = output

        out = combo[[stock,'predict','predicted']].rename(columns = {stock:'Today','predict':'Tomorow Actual','predicted':'Tomorow Pred'})

        todo = []
        correct = []
        for i in range(0,len(out)):
            row = out.iloc[i]
            if row['Tomorow Pred'] > row['Today']: 
                todo.append('Hold')
            else: todo.append('Sell')

            if math.isnan(row['Tomorow Actual']): correct.append(np.nan)
            elif row['Tomorow Actual'] > row['Today']: 
                correct.append('Hold')
            else: correct.append('Sell')
        out['todo'] = todo
        out['correct'] = correct
        outs[stock] = out
    return outs


In [184]:
length = 365
length_ = length+1

all_dates = combo.index
frame = all_dates[-length_:-1]

plan={}
for stock in stock_name:
    plan[stock] = []
i=0
for date in frame:
    available = combo.loc[:date].copy()
    outs = do_model(available,stock_name)
    for stock in stock_name:
        the_move = outs[stock].todo[-1]
        plan[stock].append(the_move)
    i+=1
    print(f'{i}/{length}', end="\r", flush=True)


        


In [ ]:
def testing(plan,stock_name,initial):
    compare_list=[]

    combo = make_stock_data(stock_name)
    for stock in stock_name:
        x = initial
        x2 = x
        x3 = x
        plann=[]
        hold=[]
        perfect=[]
        stock_plan = plan[stock]
        trimmed_combo = combo[-len(stock_plan):].copy()
        trimmed_combo['plan'] = stock_plan
        date = [x.to_timestamp() for x in trimmed_combo.index]
        for i in range(0,len(trimmed_combo)-1):
            now = trimmed_combo.iloc[i]
            tom = trimmed_combo.iloc[i+1]

            change = ((tom[stock]-now[stock])/now[stock])

            # using system
            if now.plan == 'Hold':
                x = x + (x*change)
            # just holding
            x2 = x2+(x2*change)
            # if it was perfect
            if change >0:
                x3 = x3+(x3*change)

            plann.append(x)
            hold.append(x2)
            perfect.append(x3)

        print(f'{stock}:')
        print(f'    -Perfect: {round(x3,2)}$ -- {round(((x3-initial)/initial)*100,2)}%')
        print(f'    -Plan: {round(x,2)}$ ----- {round(((x-initial)/initial)*100,2)}%')
        print(f'    -Hold: {round(x2,2)}$ ----- {round(((x2-initial)/initial)*100,2)}%')

        plan_frame = pd.DataFrame(zip(date,plann,['plan']*len(plann),[stock]*len(plann)),columns = ['date','val','type','stock'])
        hold_frame = pd.DataFrame(zip(date,hold,['hold']*len(plann),[stock]*len(plann)),columns = ['date','val','type','stock'])
        perfect_frame = pd.DataFrame(zip(date,perfect,['perfect']*len(plann),[stock]*len(plann)),columns = ['date','val','type','stock'])

        compare= pd.concat([plan_frame,hold_frame,perfect_frame])
        compare_list.append(compare)
    all_compare = pd.concat(compare_list)

    all_compare['indicator'] = all_compare['type']+' '+all_compare['stock']
    plot = px.line(x=all_compare.date,y=all_compare.val,color=all_compare.indicator)
    all_compare = all_compare.drop('indicator',axis=1)
    return all_compare,plot

    

In [ ]:
all_comp,plot = testing(plan,stock_name,1000)

pcar:
    -Perfect: 106.29$ -- 6.29%
    -Plan: 104.81$ ----- 4.81%
    -Hold: 104.24$ ----- 4.24%
amzn:
    -Perfect: 105.78$ -- 5.78%
    -Plan: 99.76$ ----- -0.24%
    -Hold: 101.28$ ----- 1.28%
tsla:
    -Perfect: 101.72$ -- 1.72%
    -Plan: 91.39$ ----- -8.61%
    -Hold: 85.67$ ----- -14.33%
oil:
    -Perfect: 108.6$ -- 8.6%
    -Plan: 108.19$ ----- 8.19%
    -Hold: 105.79$ ----- 5.79%
aapl:
    -Perfect: 103.08$ -- 3.08%
    -Plan: 103.08$ ----- 3.08%
    -Hold: 93.87$ ----- -6.13%
[Timestamp('2022-09-27 00:00:00'), Timestamp('2022-09-28 00:00:00'), Timestamp('2022-09-29 00:00:00'), Timestamp('2022-09-30 00:00:00'), Timestamp('2022-10-03 00:00:00')]


In [ ]:
plot